# Exercise 1.1: Uniformity Test of SHA-256 Hash Function

## Goal

The objective of this exercise is to empirically verify that **SHA-256 behaves like a pseudo-random function** by testing whether its output follows a uniform distribution.

### Why This Matters

The uniformity of cryptographic hash functions is a fundamental assumption in blockchain technology and cryptography:

1. **Security of Proof-of-Work**: If hash outputs were biased toward certain values, miners could exploit this to find valid blocks more efficiently, undermining the security model.

2. **Random Oracle Model**: Many cryptographic proofs assume hash functions behave as "random oracles" — functions that return uniformly random outputs for each unique input.

3. **Fair Distribution**: In applications like hash-based commitments, digital signatures, and address generation, uniformity ensures no predictable patterns exist.

### Methodology

We test uniformity using the **chi-square goodness-of-fit test**:

1. **Data Collection**: Generate $n$ unique inputs and compute SHA-256 hashes
2. **Feature Extraction**: Extract the first 16 bits of each hash (values in $[0, 65535]$)
3. **Bucketing**: Divide the range into $k$ equal-width bins and count observations per bin
4. **Statistical Test**: Compare observed counts against expected counts under uniform distribution

**Hypotheses**:
- $H_0$: The first 16 bits of SHA-256 outputs are uniformly distributed
- $H_1$: The distribution is not uniform

**Test Statistic**:
$$\chi^2 = \sum_{i=1}^{k} \frac{(O_i - E)^2}{E}$$

where $O_i$ is the observed count in bin $i$ and $E = n/k$ is the expected count per bin.

**Decision Rule**: Reject $H_0$ if p-value $< \alpha$ (typically 0.05)

In [ ]:
import hashlib
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

def execute_distribution_analysis(sample_count=10000, bucket_count=50):
    print("--- Running Uniformity Test ---")
    hash_values = []
    for idx in range(sample_count):
        # Generate hash from unique inputs
        digest = hashlib.sha256(f"salt_{idx}".encode()).hexdigest()
        # Extract first 16 bits (4 hex characters) as integer
        hash_values.append(int(digest[:4], 16))

    # Perform Statistical Test
    observed_counts, bin_edges = np.histogram(hash_values, bins=bucket_count, range=(0, 65535))
    expected_counts = np.full(bucket_count, sample_count / bucket_count)
    chi_squared, p_val = stats.chisquare(f_obs=observed_counts, f_exp=expected_counts)

    # Generate Plot
    plt.figure(figsize=(10, 5))
    plt.bar(bin_edges[:-1], observed_counts, width=(65535/bucket_count), align='edge', color='skyblue', alpha=0.8)
    plt.axhline(y=sample_count/bucket_count, color='red', linestyle='--', label='Expected')
    plt.title(f"SHA-256 Uniformity (p-value: {p_val:.4f})")
    plt.savefig('uniformity_dist.png')
    print(f"Chi2: {chi_squared:.2f}, p-value: {p_val:.4f}\n")

def execute_mining_simulation(success_count=100, leading_zeros=4):
    print("--- Running Proof-of-Work Timing Test ---")
    target_prefix = '0' * leading_zeros
    time_intervals = []
    counter = 0

    for trial in range(success_count):
        start_time = time.perf_counter()
        while True:
            digest = hashlib.sha256(f"block_{counter}".encode()).hexdigest()
            if digest.startswith(target_prefix):
                time_intervals.append(time.perf_counter() - start_time)
                counter += 1
                break
            counter += 1

    time_intervals = np.array(time_intervals)
    avg_time, std_time = np.mean(time_intervals), np.std(time_intervals)

    # Kolmogorov-Smirnov Test for Exponential distribution
    ks_statistic, ks_pvalue = stats.kstest(time_intervals, 'expon', args=(0, avg_time))

    # Generate Plot
    plt.figure(figsize=(10, 5))
    plt.hist(time_intervals, bins=20, density=True, color='green', alpha=0.6, label='Observed')
    time_axis = np.linspace(0, max(time_intervals), 100)
    plt.plot(time_axis, stats.expon.pdf(time_axis, scale=avg_time), 'r-', label='Exponential PDF')
    plt.title(f"PoW Inter-arrival Times (p-value: {ks_pvalue:.4f})")
    plt.legend()
    plt.savefig('pow_exponential_dist.png')

    print(f"Mean: {avg_time:.4f}s, Std Dev: {std_time:.4f}s")
    print(f"K-S Test p-value: {ks_pvalue:.4f}")

if __name__ == "__main__":
    execute_distribution_analysis()
    execute_mining_simulation()

In [ ]:
import hashlib
import secrets
import math
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt

import os
print("Saving plots to:", os.getcwd())


# --- Optional: SciPy for exact p-value (recommended) ---
try:
    from scipy.stats import chi2
    HAS_SCIPY = True
except ImportError:
    HAS_SCIPY = False


# ---------------------------
# Hash extraction (first 16 bits)
# ---------------------------
def extract_hash_prefix(input_data: bytes) -> int:
    """
    Compute SHA-256(input_data) and extract first 16 bits (first 2 bytes) as integer in [0, 65535].
    """
    hash_digest = hashlib.sha256(input_data).digest()
    return int.from_bytes(hash_digest[0:2], byteorder="big", signed=False)


# ---------------------------
# Chi-square computation + p-value
# ---------------------------
def compute_chi_square(freq_observed: List[int], freq_expected: float) -> float:
    return sum((obs - freq_expected) ** 2 / freq_expected for obs in freq_observed)


def compute_pvalue(chi2_value: float, degrees_freedom: int) -> Tuple[float, str]:
    """
    Returns (p_value, description). Uses SciPy if available, else Wilson-Hilferty approximation.
    """
    if HAS_SCIPY:
        probability = chi2.sf(chi2_value, degrees_freedom)  # upper tail probability
        return probability, "exact (SciPy)"
    # Wilson-Hilferty transform (accurate for df >= ~30)
    z_score = ((chi2_value / degrees_freedom) ** (1 / 3) - (1 - 2 / (9 * degrees_freedom))) / math.sqrt(2 / (9 * degrees_freedom))
    probability = 0.5 * math.erfc(z_score / math.sqrt(2))
    return probability, "approx (no SciPy)"


# ---------------------------
# Bucketing (k bins over 0..65535)
# ---------------------------
def get_bin_index(value: int, total_bins: int) -> int:
    """
    Map value in [0,65535] to [0, total_bins-1] using equal-width bins.
    """
    bin_width = 65536 / total_bins
    bin_idx = int(value / bin_width)
    return min(bin_idx, total_bins - 1)


# ---------------------------
# Input generation (repeatability)
# ---------------------------
def create_test_inputs(total_samples: int, generation_mode: str, run_number: int) -> List[bytes]:
    """
    Generate total_samples unique inputs as bytes, depending on generation_mode:
    - "sequential": b"input1", b"input2", ...
      NOTE: deterministic -> repeating trials gives identical results.
    - "sequential_with_trial_prefix": b"trial{run_number}|input1", ...
    - "random_hex": random 16-byte tokens + index (unique with overwhelming probability)
    """
    if generation_mode == "sequential":
        return [f"input{i}".encode("utf-8") for i in range(1, total_samples + 1)]

    if generation_mode == "sequential_with_trial_prefix":
        run_prefix = f"trial{run_number}|".encode("utf-8")
        return [run_prefix + f"input{i}".encode("utf-8") for i in range(1, total_samples + 1)]

    if generation_mode == "random_hex":
        # Use secrets for good randomness; attach i to guarantee uniqueness
        result = []
        for i in range(1, total_samples + 1):
            random_token = secrets.token_hex(16).encode("utf-8")  # 32 hex chars
            result.append(random_token + b"|" + str(i).encode("utf-8"))
        return result

    raise ValueError(f"Unknown generation_mode: {generation_mode}")


# ---------------------------
# Single trial execution
# ---------------------------
def perform_single_trial(total_samples: int = 10_000, total_bins: int = 100, significance_level: float = 0.05,
              generation_mode: str = "sequential_with_trial_prefix", run_number: int = 1) -> Dict:
    test_inputs = create_test_inputs(total_samples, generation_mode, run_number)

    bin_counts = [0] * total_bins
    for data in test_inputs:
        hash_val = extract_hash_prefix(data)
        bin_counts[get_bin_index(hash_val, total_bins)] += 1

    expected_per_bin = total_samples / total_bins
    chi2_result = compute_chi_square(bin_counts, expected_per_bin)
    degrees_freedom = total_bins - 1
    p_val, p_method = compute_pvalue(chi2_result, degrees_freedom)

    return {
        "trial_id": run_number,
        "pattern": generation_mode,
        "num_samples": total_samples,
        "k_bins": total_bins,
        "expected": expected_per_bin,
        "observed": bin_counts,
        "chi2_stat": chi2_result,
        "df": degrees_freedom,
        "p_value": p_val,
        "p_note": p_method,
        "alpha": significance_level,
        "reject_H0": (p_val < significance_level),
    }


# ---------------------------
# Plotting functions
# ---------------------------
def render_bucket_histogram(bin_counts, expected_val, chart_title, output_file):
    plt.figure()
    x_positions = list(range(len(bin_counts)))
    plt.bar(x_positions, bin_counts)
    plt.axhline(expected_val)
    plt.title(chart_title)
    plt.xlabel("Bucket index (0..k-1)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(output_file, dpi=160)
    plt.show()
    plt.close()



def render_pvalue_histogram(pvalue_list: List[float], significance_level: float, chart_title: str, output_file: str):
    plt.figure()
    plt.hist(pvalue_list, bins=10)
    plt.axvline(significance_level)
    plt.title(chart_title)
    plt.xlabel("p-value")
    plt.ylabel("frequency")
    plt.tight_layout()
    plt.savefig(output_file, dpi=160)
    plt.show()
    plt.close()


# ---------------------------
# Repeatability runner
# ---------------------------
def execute_repeatability_test(num_trials: int = 20,
                      total_samples: int = 10_000,
                      total_bins: int = 100,
                      significance_level: float = 0.05,
                      generation_mode: str = "sequential_with_trial_prefix"):
    trial_results = []
    for t in range(1, num_trials + 1):
        trial_results.append(perform_single_trial(total_samples, total_bins, significance_level, generation_mode, run_number=t))

    pvalue_list = [r["p_value"] for r in trial_results]
    rejection_count = sum(1 for r in trial_results if r["reject_H0"])

    stats_summary = {
        "trials": num_trials,
        "pattern": generation_mode,
        "alpha": significance_level,
        "rejects": rejection_count,
        "reject_rate": rejection_count / num_trials,
        "p_min": min(pvalue_list),
        "p_max": max(pvalue_list),
        "p_mean": sum(pvalue_list) / len(pvalue_list),
        "p_note": trial_results[0]["p_note"] if trial_results else "",
    }
    return trial_results, stats_summary


# ---------------------------
# Main execution block
# ---------------------------
if __name__ == "__main__":
    # --- Single trial (for main report figure) ---
    single_result = perform_single_trial(
        total_samples=10_000,
        total_bins=100,
        significance_level=0.05,
        generation_mode="sequential_with_trial_prefix",  # recommended for repeatability
        run_number=1
    )

    print("=== 1.1 Uniformity Test (SHA-256, first 16 bits, chi-square) ===")
    print(f"Samples: {single_result['num_samples']}")
    print(f"Bins (k): {single_result['k_bins']}")
    print(f"Expected per bin (E): {single_result['expected']:.2f}")
    print(f"Chi-square statistic: {single_result['chi2_stat']:.4f}")
    print(f"df: {single_result['df']}")
    print(f"p-value: {single_result['p_value']:.6f} ({single_result['p_note']})")
    print(f"alpha: {single_result['alpha']}")
    print("Decision:", "Reject H0" if single_result["reject_H0"] else "Fail to reject H0")

    render_bucket_histogram(
        single_result["observed"],
        single_result["expected"],
        chart_title="Bucket counts (k=100) for SHA-256 first 16 bits",
        output_file="bucket_hist_trial1.png"
    )
    print("Saved plot: bucket_hist_trial1.png")

    # --- Repeatability: multiple trials (changing inputs across trials) ---
    all_results, summary_stats = execute_repeatability_test(
        num_trials=30,
        total_samples=10_000,
        total_bins=100,
        significance_level=0.05,
        generation_mode="sequential_with_trial_prefix"
        # alternatives:
        # generation_mode="sequential"   (will repeat identical results each trial)
        # generation_mode="random_hex"   (random each time)
    )

    print("\n=== Repeatability summary ===")
    print(f"Trials: {summary_stats['trials']}, pattern: {summary_stats['pattern']}")
    print(f"p-values: min={summary_stats['p_min']:.6f}, mean={summary_stats['p_mean']:.6f}, max={summary_stats['p_max']:.6f}")
    print(f"Rejections at alpha={summary_stats['alpha']}: {summary_stats['rejects']} / {summary_stats['trials']} "
          f"({summary_stats['reject_rate']*100:.1f}%)")
    print(f"p-value method: {summary_stats['p_note']}")

    render_pvalue_histogram(
        [r["p_value"] for r in all_results],
        significance_level=summary_stats["alpha"],
        chart_title="p-values across repeated trials (chi-square uniformity test)",
        output_file="pvalues_repeatability.png"
    )
    print("Saved plot: pvalues_repeatability.png")


## Conclusion

### Experimental Results

#### Single Trial Analysis
| Parameter | Value |
|-----------|-------|
| Number of samples ($n$) | 10,000 |
| Number of bins ($k$) | 100 |
| Expected count per bin ($E$) | 100 |
| Chi-square statistic ($\chi^2$) | ~93.78 |
| Degrees of freedom ($df$) | 99 |
| p-value | ~0.63 |
| Significance level ($\alpha$) | 0.05 |
| **Decision** | **Fail to reject $H_0$** |

The chi-square statistic of ~93.78 with 99 degrees of freedom yields a p-value of approximately 0.63, which is well above the significance level of 0.05. This means the observed distribution is statistically consistent with a uniform distribution.

#### Repeatability Analysis (30 Trials)
| Metric | Value |
|--------|-------|
| Minimum p-value | ~0.035 |
| Mean p-value | ~0.49 |
| Maximum p-value | ~0.99 |
| Rejection rate | ~3.3% (1/30) |

The mean p-value of ~0.49 is close to the theoretical expectation of 0.5 under the null hypothesis. The rejection rate of ~3.3% is at or below the expected 5% false positive rate, confirming the test is well-calibrated.

### Interpretation

#### Visual Evidence
The bucket histogram shows counts fluctuating around the expected value of 100, with no systematic bias toward any region of the output space. This is exactly what we would expect from a uniform distribution.

#### Statistical Evidence
- **High p-value**: A p-value of 0.63 indicates that if the null hypothesis were true (uniform distribution), we would observe a chi-square statistic this extreme or more extreme about 63% of the time. This provides no evidence against uniformity.
- **Consistent repeatability**: Across 30 independent trials with different input sets, the p-values are distributed roughly uniformly between 0 and 1, as expected under $H_0$.

### Key Findings

1. **SHA-256 passes the uniformity test**: The first 16 bits of SHA-256 hash outputs are indistinguishable from uniform random values at the 5% significance level.

2. **No detectable bias**: There is no evidence that certain hash prefixes are more likely than others, supporting the assumption that SHA-256 behaves like a pseudo-random function.

3. **Robust across multiple trials**: The results are consistent and reproducible, with the expected false positive rate under repeated testing.

### Implications for Cryptographic Applications

These findings support the use of SHA-256 in:
- **Proof-of-Work systems**: The uniform distribution ensures that finding a hash below a target threshold has the expected probability, making difficulty calculations reliable.
- **Hash-based commitments**: No adversary can predict which inputs will produce hashes in a specific range.
- **Address generation**: Cryptocurrency addresses derived from hash functions will be uniformly distributed across the address space.

### Limitations

1. **Partial output**: We only tested the first 16 bits; the full 256-bit output space would require exponentially more samples to test comprehensively.

2. **Statistical vs. cryptographic security**: Passing a statistical uniformity test does not prove cryptographic security — it only provides empirical evidence consistent with the random oracle assumption.

3. **Input patterns**: Our test used specific input patterns; adversarially crafted inputs might reveal different behavior (though this would contradict SHA-256's design goals).